In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("GPU is not available. Please check your Colab settings.")

GPU is available!


In [ ]:
!pip install torch torchvision transformers opencv-python scikit-image matplotlib diffusers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 81.6 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision transformers opencv-python scikit-image matplotlib diffusers

In [ ]:
!pip install --upgrade transformers diffusers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4


In [ ]:
pip install torch torchvision transformers opencv-python scikit-image matplotlib

In [ ]:
# Step 1: Install Necessary Libraries
!pip install torch torchvision diffusers transformers opencv-python scikit-image matplotlib

# Step 2: Import Required Libraries
import torch
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import numpy as np
import cv2
from skimage import io
import matplotlib.pyplot as plt
from google.colab import files

# Step 3: Upload Images
print("Please upload the source face image.")
source_face_upload = files.upload()

print("Please upload the target body image.")
target_body_upload = files.upload()

# Extract file names
source_face_path = list(source_face_upload.keys())[0]
target_body_path = list(target_body_upload.keys())[0]

# Step 4: Load the Images
def load_images(source_face_path, target_body_path):
    source_face = Image.open(source_face_path).convert("RGB")
    target_body = Image.open(target_body_path).convert("RGB")
    return source_face, target_body

source_face, target_body = load_images(source_face_path, target_body_path)

# Step 5: Preprocess Images
def preprocess_images(source_face, target_body, face_position=(50, 50)):
    target_body_np = np.array(target_body)
    target_face_region = target_body_np[face_position[1]:face_position[1] + source_face.size[1],
                                        face_position[0]:face_position[0] + source_face.size[0]]

    resized_face = source_face.resize((target_face_region.shape[1], target_face_region.shape[0]))

    return resized_face, target_body_np, face_position

resized_face, target_body_np, face_position = preprocess_images(source_face, target_body)

# Step 6: Apply Diffusion Model
def apply_diffusion_model(resized_face, target_body_np, face_position):
    # Convert the target body back to PIL for the pipeline
    target_body_pil = Image.fromarray(target_body_np.astype(np.uint8)) # Ensure the array is of type uint8 before conversion

    # Load the inpainting pipeline
    pipe = StableDiffusionInpaintPipeline.from_pretrained("stabilityai/stable-diffusion-2-inpainting")
    pipe = pipe.to("cuda")

    # Convert images to suitable format
    mask = np.zeros_like(target_body_np[:, :, 0])
    mask[face_position[1]:face_position[1] + resized_face.size[1],
         face_position[0]:face_position[0] + resized_face.size[0]] = 1

    # Convert to PIL Image, ensuring the array is of type uint8
    mask_pil = Image.fromarray((mask * 255).astype(np.uint8))

    # Apply the model
    output = pipe(prompt="", image=target_body_pil, mask_image=mask_pil).images[0]

    return output

output = apply_diffusion_model(resized_face, target_body_np, face_position)

# Step 7: Blend Images Using SeamlessClone
def blend_images_seamless(output, resized_face, target_body_np, face_position):
    output_np = np.array(output)

    # Create a binary mask for the face region
    face_mask = 255 * np.ones(resized_face.size, dtype=np.uint8) # Use np.uint8 as the data type for the mask

    # Calculate the center of the face region in the target image
    center = (face_position[0] + resized_face.size[0] // 2, face_position[1] + resized_face.size[1] // 2)

    # Ensure the face position and size are within the target image bounds
    x1 = max(0, face_position[0])
    y1 = max(0, face_position[1])
    x2 = min(target_body_np.shape[1], x1 + resized_face.size[0])
    y2 = min(target_body_np.shape[0], y1 + resized_face.size[1])

    # Adjust the center accordingly
    center = ((x1 + x2) // 2, (y1 + y2) // 2)

    # Crop the face and mask to fit within the bounds
    resized_face_cropped = resized_face.crop((0, 0, x2 - x1, y2 - y1))
    face_mask_cropped = face_mask[y1:y2, x1:x2]

    # Perform seamless cloning with the adjusted parameters
    blended_image = cv2.seamlessClone(np.array(resized_face_cropped), target_body_np, np.array(face_mask_cropped), center, cv2.NORMAL_CLONE)

    return Image.fromarray(blended_image)

final_image = blend_images_seamless(output, resized_face, target_body_np, face_position)

# Step 8: Save and Display the Final Image
final_image.save("output_face_swap.jpg")
final_image.show()

# Optional: Download the final image
files.download("output_face_swap.jpg")




Please upload the source face image.


Saving Passport Size Photograph.jpg to Passport Size Photograph (6).jpg
Please upload the target body image.


Saving images.jpeg to images (6).jpeg


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


  0%|          | 0/50 [00:00<?, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>